# Mapping migration

Introduction to vector data operations

## STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [3]:
%store -r

In [38]:
#Import my libraries
import os
import pathlib
import csv  

import pandas as pd
import geopandas as gpd



## STEP 1: Create a directory for your data

For this challenge, you will need to download some data to the computer
you’re working on. We suggest using the `earthpy` library we develop to
manage your downloads, since it encapsulates many best practices as far
as:

1.  Where to store your data
2.  Dealing with archived data like .zip files
3.  Avoiding version control problems
4.  Making sure your code works cross-platform
5.  Avoiding duplicate downloads

If you’re working on one of our assignments through GitHub Classroom, it
also lets us build in some handy defaults so that you can see your data
files while you work.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Create a project folder</div></div><div class="callout-body-container callout-body"><p>The code below will help you get started with making a project
directory</p>
<ol type="1">
<li>Replace <code>project_title</code> with the actual project title,
Veery Migration 2023</li>
<li>Replace <code>'your-project-directory-name-here'</code> with a
<strong>descriptive</strong> name</li>
<li>The code should have printed out the path to your data files. Check
that your data directory exists and has data in it using the terminal or
your Finder/File Explorer.</li>
</ol></div></div>

> **File structure**
>
> These days, a lot of people find your file by searching for them or
> selecting from a `Bookmarks` or `Recents` list. Even if you don’t use
> it, your computer also keeps files in a **tree** structure of folders.
> Put another way, you can organize and find files by travelling along a
> unique **path**, e.g. `My Drive` \> `Documents` \>
> `My awesome project` \> `A project file` where each subsequent folder
> is **inside** the previous one. This is convenient because all the
> files for a project can be in the same place, and both people and
> computers can rapidly locate files they want, provided they remember
> the path.
>
> You may notice that when Python prints out a file path like this, the
> folder names are **separated** by a `/` or `\` (depending on your
> operating system). This character is called the **file separator**,
> and it tells you that the next piece of the path is **inside** the
> previous one.

In [11]:
# Create data directory
data_dir = os.path.join(
    #home directory
    pathlib.Path.home(),
    #earth data analytics directory
    'earth-analytics',
    'data',
    #project directory
    'migration',
)
print(data_dir)

os.makedirs(data_dir, exist_ok=True)
# Download sample data
#data_dir.get_data()

# Display the project directory
#project.project_dir

/Users/gml/earth-analytics/data/migration


## STEP 2: Define your study area – the ecoregions of North America

Your sample data package included a Shapefile of global **ecoregions**.
You should be able to see changes in the number of observations of Veery
thrush in each ecoregion throughout the year.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-info"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div></div><div class="callout-body-container callout-body"><p>You don’t have to use ecoregions to group species observations – you
could choose to use political boundaries like countries or states, other
natural boundaries like watersheds, or even uniform hexagonal areas as
is common in conservation work. We chose ecoregions because we expect
the suitability for a species at a particular time of year to be
relatively consistent across the region.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>The ecoregion data will be available as a <strong>shapefile</strong>.
Learn more about shapefiles and vector data in this <a
href="https://www.earthdatascience.org/courses/intro-to-earth-data-science/file-formats/use-spatial-data/use-vector-data/">Introduction
to Spatial Vector Data File Formats in Open Source Python</a></p></div></div>

### Load the ecoregions into Python

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Load ecoregions into Python</div></div><div class="callout-body-container callout-body"><p>Download and save ecoregion boundaries from the EPA:</p>
<ol type="1">
<li>Replace <code>a_path</code> with the path your created for your
ecoregions file.</li>
<li>Make a quick plot with <code>.plot()</code> to make sure the
download worked.</li>
</ol></div></div>

In [28]:
eco_url = (
    "https://storage.googleapis.com/"
    "teow2016/Ecoregions2017.zip"
)

# Open up the ecoregions boundaries
eco_dir = os.path.join(data_dir, 'resolve_ecoregions')
#make directory
os.makedirs(eco_dir, exist_ok=True)

#join ecoregions shapefile
eco_path = os.path.join(eco_dir, 'ecoregions.shp')

# Only download and extract if not already saved
if not os.path.exists(eco_path):
    gdf = gpd.read_file(f"zip+{eco_url}")  # read directly from zip URL
    gdf.to_file(eco_path)
    print("Downloaded and saved shapefile to:", eco_path)
else:
    gdf = gpd.read_file(eco_path)
    print("Loaded existing shapefile from:", eco_path)

Downloaded and saved shapefile to: /Users/gml/earth-analytics/data/migration/resolve_ecoregions/ecoregions.shp


## STEP 3: Load species observation data

For this challenge, you will use a database called the [Global
Biodiversity Information Facility (GBIF)](https://www.gbif.org/). GBIF
is compiled from species observation data all over the world, and
includes everything from museum specimens to photos taken by citizen
scientists in their backyards. We’ve compiled some sample data in the
same format that you will get from GBIF.

Let’s start by looking at a little of the raw data.

In [ ]:
gbif_path = '/Users/gml/earth-analytics/data/migration/gbif_veery.csv'

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Load GBIF data</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Look at the beginning of the file you downloaded using the code
below. What do you think the <strong>delimiter</strong> is?</li>
<li>Run the following code cell. What happens?</li>
<li>Uncomment and modify the parameters of <code>pd.read_csv()</code>
below until your data loads successfully and you have only the columns
you want.</li>
</ol></div></div>

You can use the following code to look at the beginning of your file:

In [42]:
!head -n 2 $gbif_path 

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
4158712344	8a863029-f435-446a-821e-275f4f641165	https://observation.org/observation/273993634	Animalia	Chordata	Aves	Passeriformes	Turdidae	Catharus	Catharus fuscescens		SPECIES	Catharus fuscescens (Stephens, 1817)	Catharus fuscescens		CA	Ontario - Point Pelee NP	Ontario	PRESENT	1	c8d737e0-2ff8-42e8-b8fc-6b805d26fc5f	41.9137	-82.5092	8.0						2023-05-19	19	5	202

In [40]:
gbif_df = pd.read_csv(
    gbif_path,
    sep="\t",                # tab-separated
    engine="c",              # default engine (supports low_memory)
    low_memory=False,        # keeps column types consistent
    quoting=csv.QUOTE_NONE,  # prevents pandas from misreading quotes
    na_values=["", "NA", "NaN", "\\N"],  # treat blanks as NaN
    parse_dates=["eventDate", "lastInterpreted"],  # optional
)

gbif_df.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,4158712344,8a863029-f435-446a-821e-275f4f641165,https://observation.org/observation/273993634,Animalia,Chordata,Aves,Passeriformes,Turdidae,Catharus,Catharus fuscescens,...,NaN,NaN,CC_BY_NC_4_0,Stichting Observation International,User 1751,NaN,NaN,2025-09-24T01:21:44.728Z,NaN,NaN
1,4923515059,36f15a36-6b45-442e-9c31-cd633423aee0,80de1f4e-30b8-4ffc-88e7-654a025164a5,Animalia,Chordata,Aves,Passeriformes,Turdidae,Catharus,Catharus fuscescens,...,NaN,NaN,CC0_1_0,The Field Museum of Natural History,D. E. Willard,NaN,NaN,2025-09-19T13:12:11.782Z,NaN,COORDINATE_ROUNDED;INSTITUTION_MATCH_FUZZY;COL...
2,4923522410,36f15a36-6b45-442e-9c31-cd633423aee0,da8061c1-aecd-49f8-84a7-31c39358f2cf,Animalia,Chordata,Aves,Passeriformes,Turdidae,Catharus,Catharus fuscescens,...,NaN,NaN,CC0_1_0,The Field Museum of Natural History,D. E. Willard,NaN,NaN,2025-09-19T13:12:11.782Z,NaN,COORDINATE_ROUNDED;INSTITUTION_MATCH_FUZZY;COL...
3,4923520798,36f15a36-6b45-442e-9c31-cd633423aee0,bf61d7c8-b847-4250-8d5a-798694a2ddfc,Animalia,Chordata,Aves,Passeriformes,Turdidae,Catharus,Catharus fuscescens,...,NaN,NaN,CC0_1_0,The Field Museum of Natural History,D. E. Willard,NaN,NaN,2025-09-19T13:12:11.782Z,NaN,COORDINATE_ROUNDED;INSTITUTION_MATCH_FUZZY;COL...
4,4923520314,36f15a36-6b45-442e-9c31-cd633423aee0,665d4e55-daaa-40d5-8339-e4b31bb303f4,Animalia,Chordata,Aves,Passeriformes,Turdidae,Catharus,Catharus fuscescens,...,NaN,NaN,CC0_1_0,The Field Museum of Natural History,J. Steadham,NaN,NaN,2025-09-19T13:12:11.782Z,NaN,INSTITUTION_MATCH_FUZZY;COLLECTION_MATCH_FUZZY


### Convert the GBIF data to a GeoDataFrame

To plot the GBIF data, we need to convert it to a `GeoDataFrame` first.
This will make some special geospatial operations from `geopandas`
available, such as spatial joins and plotting.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Convert `DataFrame` to `GeoDataFrame`</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Replace <code>your_dataframe</code> with the name of the
<code>DataFrame</code> you just got from GBIF</li>
<li>Replace <code>longitude_column_name</code> and
<code>latitude_column_name</code> with column names from your
`DataFrame</li>
<li>Run the code to get a <code>GeoDataFrame</code> of the GBIF
data.</li>
</ol></div></div>

In [45]:
print(gbif_df.decimalLongitude)
print(gbif_df.decimalLatitude)


0        -82.509200
1        -87.611721
2        -87.611721
3        -87.611721
4        -87.630134
            ...    
165800   -75.125400
165801   -89.804900
165802   -82.475300
165803   -74.546800
165804   -75.125400
Name: decimalLongitude, Length: 165805, dtype: float64
0         41.913700
1         41.852812
2         41.852812
3         41.852812
4         41.880356
            ...    
165800    40.062600
165801    44.696700
165802    42.047800
165803    46.163800
165804    40.062600
Name: decimalLatitude, Length: 165805, dtype: float64


In [47]:
gbif_gdf = (
    gpd.GeoDataFrame(
        gbif_df, 
        geometry=gpd.points_from_xy(
            gbif_df.decimalLongitude, 
            gbif_df.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['month','geometry']]
)
gbif_gdf

,month,geometry
0,5,POINT (-82.5092 41.9137)
1,5,POINT (-87.61172 41.85281)
2,9,POINT (-87.61172 41.85281)
3,5,POINT (-87.61172 41.85281)
4,9,POINT (-87.63013 41.88036)
...,...,...
165800,9,POINT (-75.1254 40.0626)
165801,6,POINT (-89.8049 44.6967)
165802,5,POINT (-82.4753 42.0478)
165803,6,POINT (-74.5468 46.1638)


<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Make sure to store the new version of your <code>DataFrame</code> for
other notebooks!</p>
<div id="f1412915" class="cell" data-execution_count="14">
<div class="sourceCode" id="cb1"><pre
class="sourceCode python cell-code"><code class="sourceCode python"><span id="cb1-1"><a href="#cb1-1" aria-hidden="true" tabindex="-1"></a><span class="op">%</span>store ecoregions_gdf gbif_gdf</span></code></pre></div>
</div></div></div>

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [52]:
%store gdf gbif_gdf

Stored 'gdf' (GeoDataFrame)
Stored 'gbif_gdf' (GeoDataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!